In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import ipywidgets as widgets
from IPython.display import display

## Carga y Revisión de los Datos
---

In [3]:
df_clientes=pd.read_csv('clientes.csv')
df_clientes.head(10)

,id_cliente,edad,antiguedad_meses,gasto_mensual,uso_datos_gb,tipo_plan
0,1,39,119,76.63,3.60,Prepago
1,2,33,44,59.45,1.46,Postpago
2,3,41,49,107.27,21.97,Control
3,4,50,104,68.43,1.72,Control
4,5,32,75,64.60,7.36,Prepago
5,6,32,61,41.83,2.79,Prepago
6,7,50,67,55.23,1.15,Control
7,8,42,6,44.42,8.04,Control
8,9,30,9,16.69,2.03,Postpago
9,10,40,6,79.49,10.02,Postpago


In [4]:
df_ventas=pd.read_csv('ventas.csv')
df_ventas.head(10)

,id_venta,id_cliente,id_producto,fecha_venta,monto
0,1,485,6,2023-09-15,35
1,2,639,5,2023-10-09,15
2,3,986,8,2023-07-23,50
3,4,16,7,2023-09-11,60
4,5,188,9,2023-02-01,25
5,6,783,5,2023-01-05,15
6,7,459,10,2023-11-04,5
7,8,845,9,2023-06-30,25
8,9,366,2,2023-05-23,20
9,10,362,8,2023-05-07,50


In [5]:
df_productos=pd.read_csv('productos.csv')
df_productos.head(10)

,id_producto,nombre_producto,categoria,precio_unitario
0,1,Plan Prepago Básico,Prepago,10
1,2,Plan Prepago Plus,Prepago,20
2,3,Plan Postpago Ilimitado,Postpago,120
3,4,Plan Postpago Medio,Postpago,80
4,5,Paquete de Datos 3GB,Datos,15
5,6,Paquete de Datos 10GB,Datos,35
6,7,Combo Voz + Datos,Mixto,60
7,8,Servicio de Roaming,Roaming,50
8,9,Streaming Premium,Streaming,25
9,10,SMS Ilimitados,SMS,5


In [6]:
df_reclamos=pd.read_csv('reclamos.csv')
df_reclamos.head(10)

,id_reclamo,id_cliente,fecha_reclamo,tipo_reclamo,resuelto
0,1,149,2023-09-29,Cobertura,1
1,2,47,2023-04-24,Atención,1
2,3,207,2023-08-30,Portabilidad,1
3,4,676,2023-06-25,Cobertura,0
4,5,202,2023-06-17,Técnico,1
5,6,112,2023-09-28,Técnico,1
6,7,503,2023-11-28,Facturación,1
7,8,298,2023-09-01,Portabilidad,1
8,9,630,2023-05-24,Portabilidad,0
9,10,334,2023-09-03,Facturación,1


## Limpieza y transformación
---

In [7]:
# Codificar tipo_plan a valores numéricos
df_clientes['tipo_plan_cod'] = df_clientes['tipo_plan'].map({'Prepago': 0, 'Control': 1, 'Postpago': 2})

# Métricas desde la tabla de ventas
ventas_agg = df_ventas.groupby('id_cliente').agg(
    total_ventas=('id_venta', 'count'),
    gasto_total=('monto', 'sum'),
    frecuencia_compra_mensual=('id_venta', lambda x: round(len(x) / 12, 2))  # asumimos 1 año de datos
).reset_index()

# Categoría más comprada por cliente
ventas_prod = df_ventas.merge(df_productos, on='id_producto')
categoria_fav = ventas_prod.groupby('id_cliente')['categoria'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reset_index()
categoria_fav.rename(columns={'categoria': 'categoria_favorita'}, inplace=True)

# Codificación de categoría favorita
categoria_fav['categoria_fav_cod'] = categoria_fav['categoria_favorita'].astype('category').cat.codes

# Métricas desde la tabla de reclamos
reclamos_agg = df_reclamos.groupby('id_cliente').agg(
    n_reclamos=('id_reclamo', 'count'),
    reclamos_no_resueltos=('resuelto', lambda x: (x == 0).sum()),
    porcentaje_resueltos=('resuelto', lambda x: round((x.sum() / len(x)) * 100, 2))
).reset_index()

# Integrar todo con la tabla base de clientes
df_clientes_enriquecido = df_clientes.merge(ventas_agg, on='id_cliente', how='left')
df_clientes_enriquecido = df_clientes_enriquecido.merge(categoria_fav[['id_cliente', 'categoria_fav_cod']], on='id_cliente', how='left')
df_clientes_enriquecido = df_clientes_enriquecido.merge(reclamos_agg, on='id_cliente', how='left')

# Rellenar nulos (clientes sin ventas o sin reclamos)
df_clientes_enriquecido.fillna({
    'total_ventas': 0,
    'gasto_total': 0,
    'frecuencia_compra_mensual': 0,
    'categoria_fav_cod': -1,
    'n_reclamos': 0,
    'reclamos_no_resueltos': 0,
    'porcentaje_resueltos': 100
}, inplace=True)

# Mostrar tabla final
df_clientes_enriquecido.head(20)

,id_cliente,edad,antiguedad_meses,gasto_mensual,uso_datos_gb,tipo_plan,tipo_plan_cod,total_ventas,gasto_total,frecuencia_compra_mensual,categoria_fav_cod,n_reclamos,reclamos_no_resueltos,porcentaje_resueltos
0,1,39,119,76.63,3.60,Prepago,0,2.0,50.0,0.17,6.0,0.0,0.0,100.00
1,2,33,44,59.45,1.46,Postpago,2,2.0,60.0,0.17,0.0,3.0,1.0,66.67
2,3,41,49,107.27,21.97,Control,1,1.0,80.0,0.08,2.0,0.0,0.0,100.00
3,4,50,104,68.43,1.72,Control,1,3.0,70.0,0.25,0.0,0.0,0.0,100.00
4,5,32,75,64.60,7.36,Prepago,0,3.0,155.0,0.25,2.0,1.0,1.0,0.00
5,6,32,61,41.83,2.79,Prepago,0,2.0,115.0,0.17,0.0,0.0,0.0,100.00
6,7,50,67,55.23,1.15,Control,1,3.0,190.0,0.25,1.0,1.0,0.0,100.00
7,8,42,6,44.42,8.04,Control,1,3.0,45.0,0.25,3.0,0.0,0.0,100.00
8,9,30,9,16.69,2.03,Postpago,2,3.0,45.0,0.25,3.0,0.0,0.0,100.00
9,10,40,6,79.49,10.02,Postpago,2,3.0,125.0,0.25,4.0,1.0,0.0,100.00


## Normalización de variables
---

In [10]:
columnas_cluster = [
    'edad', 'antiguedad_meses', 'gasto_mensual', 'uso_datos_gb',
    'tipo_plan_cod', 'total_ventas', 'gasto_total',
    'frecuencia_compra_mensual', 'categoria_fav_cod',
    'n_reclamos', 'reclamos_no_resueltos', 'porcentaje_resueltos'
]

X_cluster = df_clientes_enriquecido[columnas_cluster].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# Mostrar los primeros valores normalizados
pd.DataFrame(X_scaled, columns=columnas_cluster).head()

,edad,antiguedad_meses,gasto_mensual,uso_datos_gb,tipo_plan_cod,total_ventas,gasto_total,frecuencia_compra_mensual,categoria_fav_cod,n_reclamos,reclamos_no_resueltos,porcentaje_resueltos
0,0.441453,1.724791,0.756280,-0.284079,-1.366894,-0.560772,-0.750215,-0.538267,2.427778,-0.893311,-0.404439,0.390395
1,-0.191757,-0.413450,-0.089632,-0.738881,0.918882,-0.560772,-0.645997,-0.538267,-0.898705,2.456606,2.076782,-0.786977
2,0.652523,-0.270901,2.264937,3.619989,-0.224006,-1.121544,-0.437560,-1.143363,0.210123,-0.893311,-0.404439,0.390395
3,1.602339,1.297143,0.352527,-0.683624,-0.224006,0.000000,-0.541778,-0.000403,-0.898705,-0.893311,-0.404439,0.390395
4,-0.297292,0.470356,0.163944,0.515012,-1.366894,0.000000,0.344076,-0.000403,0.210123,0.223328,2.076782,-3.142073


## Segmentación con K-means
---

In [15]:
# Preparamos los datos
X_cluster = df_clientes_enriquecido[[
    'edad', 'antiguedad_meses', 'gasto_mensual', 'uso_datos_gb',
    'tipo_plan_cod', 'total_ventas', 'gasto_total',
    'frecuencia_compra_mensual', 'categoria_fav_cod',
    'n_reclamos', 'reclamos_no_resueltos', 'porcentaje_resueltos'
]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Función para graficar dinámicamente
def plot_clusters(k):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    df_plot = pd.DataFrame({
        'PCA1': X_pca[:, 0],
        'PCA2': X_pca[:, 1],
        'cluster': clusters
    })
    
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df_plot, x='PCA1', y='PCA2', hue='cluster', palette='Set2', s=60, edgecolor='black')
    plt.title(f'Visualización PCA con K = {k}', fontsize=14)
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.legend(title='Cluster')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [13]:
# Aplicar KMeans con K=4
kmeans_final = KMeans(n_clusters=2, random_state=42, n_init=10)
clusters = kmeans_final.fit_predict(X_scaled)

# Agregar los clústeres al DataFrame original
df_clientes_enriquecido['cluster'] = clusters

# Ver una muestra de clientes con su clúster asignado
df_clientes_enriquecido[['id_cliente', 'edad', 'gasto_mensual', 'uso_datos_gb', 
                         'total_ventas', 'n_reclamos', 'cluster']].head(10)

,id_cliente,edad,gasto_mensual,uso_datos_gb,total_ventas,n_reclamos,cluster
0,1,39,76.63,3.60,2.0,0.0,0
1,2,33,59.45,1.46,2.0,3.0,1
2,3,41,107.27,21.97,1.0,0.0,0
3,4,50,68.43,1.72,3.0,0.0,0
4,5,32,64.60,7.36,3.0,1.0,1
5,6,32,41.83,2.79,2.0,0.0,0
6,7,50,55.23,1.15,3.0,1.0,0
7,8,42,44.42,8.04,3.0,0.0,0
8,9,30,16.69,2.03,3.0,0.0,0
9,10,40,79.49,10.02,3.0,1.0,0


## Evaluación de clústeres
---

In [14]:
from sklearn.metrics import silhouette_score

# Evaluación del clustering con Silhouette Score
sil_score = silhouette_score(X_scaled, df_clientes_enriquecido['cluster'])

print(f" Silhouette Score (K={df_clientes_enriquecido['cluster'].nunique()}): {sil_score:.4f}")

 Silhouette Score (K=2): 0.2735


## Visualización de resultados
---

In [17]:
# Crear slider interactivo
k_slider = widgets.IntSlider(value=2, min=2, max=10, step=1, description='N° Clusters (K):')
widgets.interact(plot_clusters, k=k_slider);

interactive(children=(IntSlider(value=2, description='N° Clusters (K):', max=10, min=2), Output()), _dom_class…

## Análisis e interpretación de segmentos
---

In [ ]:
# Agrupar por clúster
resumen_clusters = df_clientes_enriquecido.groupby('cluster').agg({
    'edad': 'mean',
    'antiguedad_meses': 'mean',
    'gasto_mensual': 'mean',
    'uso_datos_gb': 'mean',
    'total_ventas': 'mean',
    'gasto_total': 'mean',
    'frecuencia_compra_mensual': 'mean',
    'n_reclamos': 'mean',
    'porcentaje_resueltos': 'mean',
    'categoria_fav_cod': 'median',
    'tipo_plan_cod': 'median'
}).round(2)

resumen_clusters.head()

,edad,antiguedad_meses,gasto_mensual,uso_datos_gb,total_ventas,gasto_total,frecuencia_compra_mensual,n_reclamos,porcentaje_resueltos,categoria_fav_cod,tipo_plan_cod
cluster,,,,,,,,,,,
0,34.95,59.09,61.44,4.91,3.04,123.41,0.25,0.63,100.00,2.0,2.0
1,34.06,55.23,60.33,5.06,2.78,114.01,0.23,1.76,27.29,1.0,1.0


### Segmento 0 — Clientes jóvenes y de bajo consumo
- Perfil: Jóvenes, recién incorporados, consumen poco y tienen planes económicos (prepago o control).

- Comportamiento: Hacen pocas compras, reclaman poco y usan pocos GB.

- Oportunidades: Upselling con combos accesibles o campañas educativas (streaming, roaming, data ilimitada).

### Segmento 1 — Clientes consolidados y de alto valor
- Perfil: Mayor antigüedad, gasto elevado, usan servicios intensivos y tienen más interacción con Claro.

- Comportamiento: Compran más, reclaman más, pero se resuelven los casos. Prefieren planes postpago y servicios mixtos.

- Oportunidades: Fidelización (retención), beneficios VIP, migración a planes premium.

### Recomendaciones generales
- Segmento 0: aplicar campañas de captación y crecimiento.

- Segmento 1: estrategias de fidelización, personalización de promociones y monitoreo del churn (por alto reclamo).

## Reglas básicas de recomendación
---

### Segmento 0: Clientes de bajo consumo / prepago
- **uso_datos_gb < 3:** Sugerir paquetes de datos económicos o combos iniciales.
- **antiguedad_meses < 12:** Ofrecer bonificaciones por renovación o portabilidad para retención.
- **tipo_plan_cod == 0 (prepago):**	Promocionar planes control o postpago básicos como evolución.
- **total_ventas == 0:** Enviar campañas educativas: "Conoce nuestros beneficios al contratar planes".
- **n_reclamos == 0:** Incentivar feedback voluntario o promociones por primera compra.

### Segmento 1: Clientes consolidados / alto valor
- **uso_datos_gb > 7:**	Ofrecer planes ilimitados o superiores con beneficios exclusivos.
- **n_reclamos >= 2 y porcentaje_resueltos < 95:** Enviar contacto de soporte personalizado o asesor asignado.
- **gasto_total > 300:** Activar campañas de fidelización (puntos, descuentos, upgrades).
- **tipo_plan_cod == 2:** Sugerir servicios premium (roaming, streaming, multilínea).
- **frecuencia_compra_mensual > 1:** Ofrecer automatización de pagos o renovación de plan anticipada.

### REGLAS TRANSVERSALES
- **porcentaje_resueltos < 80:**	Monitoreo proactivo desde SAC o atención VIP.
- **categoria_fav_cod == "Streaming":**	Campañas de alianzas (ej. Netflix, Spotify, Disney+).
- **categoria_fav_cod == "Roaming":**	Activar promociones por viajes o bundles internacionales.